In [ ]:
from google.colab import auth
auth.authenticate_user()

In [ ]:
import gcsfs
import tensorflow as tf
fs = gcsfs.GCSFileSystem()
!echo "deb http://packages.cloud.google.com/apt gcsfuse-bionic main" > /etc/apt/sources.list.d/gcsfuse.list
!curl https://packages.cloud.google.com/apt/doc/apt-key.gpg | apt-key add -

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1022  100  1022    0     0  13491      0 --:--:-- --:--:-- --:--:-- 13626
OK


In [ ]:
!apt -qq update

65 packages can be upgraded. Run 'apt list --upgradable' to see them.
W: http://packages.cloud.google.com/apt/dists/gcsfuse-bionic/InRelease: Key is stored in legacy trusted.gpg keyring (/etc/apt/trusted.gpg), see the DEPRECATION section in apt-key(8) for details.
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)


In [ ]:
!apt -qq install gcsfuse

The following NEW packages will be installed:
  gcsfuse
0 upgraded, 1 newly installed, 0 to remove and 65 not upgraded.
Need to get 8,418 kB of archives.
After this operation, 0 B of additional disk space will be used.
Selecting previously unselected package gcsfuse.
(Reading database ... 123629 files and directories currently installed.)
Preparing to unpack .../gcsfuse_2.5.1_amd64.deb ...
Unpacking gcsfuse (2.5.1) ...
Setting up gcsfuse (2.5.1) ...


In [ ]:
!git clone https://github.com/ouriquco/MTR.git

Cloning into 'MTR'...
remote: Enumerating objects: 109, done.
remote: Counting objects: 100% (49/49), done.
remote: Compressing objects: 100% (42/42), done.
remote: Total 109 (delta 16), reused 7 (delta 7), pack-reused 60 (from 1)
Receiving objects: 100% (109/109), 512.73 KiB | 4.20 MiB/s, done.
Resolving deltas: 100% (23/23), done.


In [ ]:
# !mkdir ./MTR/data/waymo
!gcsfuse --implicit-dirs processed_waymo_motion_data ./MTR/data/waymo

{"timestamp":{"seconds":1731996501,"nanos":550502896},"severity":"INFO","message":"Start gcsfuse/2.5.1 (Go version go1.23.0) for app \"\" using mount point: /content/MTR/data/waymo\n"}
{"timestamp":{"seconds":1731996501,"nanos":550538900},"severity":"INFO","message":"GCSFuse config","config":{"AppName":"","CacheDir":"","Debug":{"ExitOnInvariantViolation":false,"Fuse":false,"Gcs":false,"LogMutex":false},"EnableHns":true,"FileCache":{"CacheFileForRangeRead":false,"DownloadChunkSizeMb":50,"EnableCrc":false,"EnableODirect":false,"EnableParallelDownloads":false,"MaxParallelDownloads":24,"MaxSizeMb":-1,"ParallelDownloadsPerFile":16,"WriteBufferSize":4194304},"FileSystem":{"DirMode":"755","DisableParallelDirops":false,"FileMode":"644","FuseOptions":[],"Gid":-1,"IgnoreInterrupts":true,"KernelListCacheTtlSecs":0,"RenameDirLimit":0,"TempDir":"","Uid":-1},"Foreground":false,"GcsAuth":{"AnonymousAccess":false,"KeyFile":"","ReuseTokenFromUrl":true,"TokenUrl":""},"GcsConnection":{"BillingProject":""

In [ ]:
!pip install -r ./MTR/requirements.txt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 2.1 MB/s eta 0:00:00


In [ ]:
!pip show torch

Name: torch
Version: 2.5.1+cu121
Summary: Tensors and Dynamic neural networks in Python with strong GPU acceleration
Home-page: https://pytorch.org/
Author: PyTorch Team
Author-email: packages@pytorch.org
License: BSD-3-Clause
Location: /usr/local/lib/python3.10/dist-packages
Requires: filelock, fsspec, jinja2, networkx, sympy, typing-extensions
Required-by: accelerate, fastai, peft, sentence-transformers, timm, torchaudio, torchvision


In [ ]:
# Changes
# Edit setup.py so that mtr.ops.knn becomes MTR.mtr.ops.knn and add 'MTR/mtr/version.py' in the write_version_to_file
# Edit torchrun_train.sh so that train.py become MTR/tools/train.py
# Edit train.py so that the mtr path is visible, add this near imports
# import sys
# sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath(__file__))))
!python ./MTR/setup.py develop

/usr/local/lib/python3.10/dist-packages/setuptools/dist.py:294: InformationOnly: Normalizing '0.1.0+0000000' to '0.1.0+0'
  self.metadata.version = self._normalize_version(self.metadata.version)
running develop
/usr/local/lib/python3.10/dist-packages/setuptools/command/develop.py:41: EasyInstallDeprecationWarning: easy_install command is deprecated.
!!

        ********************************************************************************
        Please avoid running ``setup.py`` and ``easy_install``.
        Instead, use pypa/build, pypa/installer or other
        standards-based tools.

        See https://github.com/pypa/setuptools/issues/917 for details.
        ********************************************************************************

!!
  easy_install.initialize_options(self)
/usr/local/lib/python3.10/dist-packages/setuptools/_distutils/cmd.py:66: SetuptoolsDeprecationWarning: setup.py install is deprecated.
!!

        ***************************************************

In [ ]:
!bash ./MTR/tools/scripts/torchrun_train.sh 1 --cfg_file ./MTR/tools/cfgs/waymo/mtr+100_percent_data.yaml --batch_size 16 --epochs 10 --extra_tag my_first_exp


+ NGPUS=1
+ PY_ARGS='--cfg_file ./MTR/tools/cfgs/waymo/mtr+100_percent_data.yaml --batch_size 16 --epochs 10 --extra_tag my_first_exp'
+ true
+ PORT=56948
++ nc -z 127.0.0.1 56948
++ echo 127
+ status=127
+ '[' 127 '!=' 0 ']'
+ break
+ echo 56948
56948
+ torchrun --nproc_per_node=1 --rdzv_endpoint=localhost:56948 MTR/tools/train.py --launcher pytorch --cfg_file ./MTR/tools/cfgs/waymo/mtr+100_percent_data.yaml --batch_size 16 --epochs 10 --extra_tag my_first_exp
2024-11-19 06:16:56,884   INFO  **********************Start logging**********************
2024-11-19 06:16:56,884   INFO  CUDA_VISIBLE_DEVICES=ALL
2024-11-19 06:16:56,884   INFO  total_batch_size: 16
2024-11-19 06:16:56,884   INFO  cfg_file         ./MTR/tools/cfgs/waymo/mtr+100_percent_data.yaml
2024-11-19 06:16:56,884   INFO  batch_size       16
2024-11-19 06:16:56,884   INFO  epochs           10
2024-11-19 06:16:56,884   INFO  workers          8
2024-11-19 06:16:56,884   INFO  extra_tag        my_first_exp
2024-11-19 06:16:56